In [2]:
import os

In [3]:
os.getcwd()

'c:\\Project\\Automatic-Image-Captioning\\research'

In [4]:
os.chdir("../")

In [ ]:
#config entity file
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    mongo_URI: str
    image_data_folder: Path
    csv_file_path: Path

In [ ]:
## config configuration
from src.ImageCaption.constants import *
from src.ImageCaption.utils.common import read_yaml, create_directory

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directory([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directory([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            mongo_URI = os.environ.get(config.mongo_uri),
            image_data_folder = config.image_data_folder,
            csv_file_path = config.csv_file_path
        )

        return data_ingestion_config

In [7]:
# components

import os
from src.ImageCaption import logger

In [ ]:
#component file
from src.ImageCaption.constants import *
def download_images_and_captions_from_mongodb(config: DataIngestionConfig):
    # Connect to MongoDB
    client = MongoClient(config.mongo_URI)
    db = client[DB_NAME]
    collection = db[COLLECTION_NAME]
    logger.info('Connecting to MongoDB...')

    # Find all documents in the collection
    documents = collection.find()

    # Prepare a list to store the caption data for CSV
    csv_data = []

    os.makedirs(config.image_data_folder, exist_ok=True)
    # Loop through the documents and save the images
    for doc in documents:
        image_name = doc['image_name']
        encoded_image = doc['image_data']
        captions = doc['captions']
        image_path = os.path.join(config.image_data_folder, image_name)

        # Decode and save the image file
        with open(image_path, 'wb') as image_file:
            image_file.write(base64.b64decode(encoded_image))
        
        # Add the captions to the CSV data list
        for caption in captions:
            csv_data.append({'image': image_name, 'caption': caption})

    # Save the CSV data to a file
    df = pd.DataFrame(csv_data)
    df.to_csv(config.csv_file_path, index=False)

    logger.info('mongo data Download complete!')

In [ ]:
#pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    download_images_and_captions_from_mongodb(data_ingestion_config)

except Exception as e:
    raise e

FileNotFoundError: [Errno 2] No such file or directory: 'config\\config.yaml'